<a href="https://colab.research.google.com/github/milkyme/CS246/blob/main/CS246_Colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 54.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=b1fc6c1564fe92e1cd3129c94ccc55bb7001dc690703523369bd21c7200bfbed
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2022-01-08 03:28:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  71.7MB/s    in 0.2s    

2022-01-08 03:28:46 (71.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [8]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [9]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [10]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [12]:
products.take(3)

[Row(product_id=1, product_name='Chocolate Sandwich Cookies', aisle_id='61', department_id='19'),
 Row(product_id=2, product_name='All-Seasons Salt', aisle_id='104', department_id='13'),
 Row(product_id=3, product_name='Robust Golden Unsweetened Oolong Tea', aisle_id='94', department_id='7')]

In [13]:
orders.take(3)

[Row(order_id=1, product_id=49302, add_to_cart_order=1, reordered=1),
 Row(order_id=1, product_id=11109, add_to_cart_order=2, reordered=1),
 Row(order_id=1, product_id=10246, add_to_cart_order=3, reordered=0)]

In [37]:
# YOUR CODE HERE
orders_with_products = orders.join(products, on = 'product_id')
orders_with_products.take(3)

[Row(product_id=49302, order_id=1, add_to_cart_order=1, reordered=1, product_name='Bulgarian Yogurt', aisle_id='120', department_id='16'),
 Row(product_id=11109, order_id=1, add_to_cart_order=2, reordered=1, product_name='Organic 4% Milk Fat Whole Milk Cottage Cheese', aisle_id='108', department_id='16'),
 Row(product_id=10246, order_id=1, add_to_cart_order=3, reordered=0, product_name='Organic Celery Hearts', aisle_id='83', department_id='4')]

In [74]:
import pyspark.sql.functions as F

In [93]:
products_for_each = orders_with_products.groupBy(['order_id']).agg(F.collect_list("product_id").alias('product_id'))
products_for_each.take(3)

[Row(order_id=1, product_id=[49302, 11109, 10246, 49683, 43633, 13176, 47209, 22035]),
 Row(order_id=96, product_id=[20574, 30391, 40706, 25610, 27966, 24489, 39275]),
 Row(order_id=112, product_id=[27104, 21174, 41860, 38273, 47209, 5876, 29217, 9047, 4549, 22425, 11776])]

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [94]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="product_id", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(products_for_each)

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [95]:
# YOUR CODE HERE
print(model.freqItemsets.count())
print(model.associationRules.count())

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


120
0


Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [96]:
# YOUR CODE HERE
fpGrowth2 = FPGrowth(itemsCol="product_id", minSupport=0.001, minConfidence=0.5)
model2 = fpGrowth2.fit(products_for_each)

In [97]:
print(model2.freqItemsets.count())
print(model2.associationRules.count())

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


4444
11


In [99]:
model2.freqItemsets.take(5)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[Row(items=[26317], freq=370),
 Row(items=[6532], freq=259),
 Row(items=[46676], freq=993),
 Row(items=[46676, 47766], freq=137),
 Row(items=[46676, 13176], freq=157)]

In [98]:
model2.associationRules.take(5)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[Row(antecedent=[4605, 16797], consequent=[24852], confidence=0.5357142857142857, lift=3.7536332219526702, support=0.0011432142612168373),
 Row(antecedent=[8174, 47209], consequent=[13176], confidence=0.5283018867924528, lift=4.477904539027839, support=0.0014937999679900007),
 Row(antecedent=[22035, 47209], consequent=[13176], confidence=0.5314685314685315, lift=4.504745125675359, support=0.0011584571180330617),
 Row(antecedent=[8174, 27966], consequent=[13176], confidence=0.5412186379928315, lift=4.587387356098284, support=0.0011508356896249496),
 Row(antecedent=[30391, 47209, 21137], consequent=[13176], confidence=0.546875, lift=4.635330870478036, support=0.0010669999771357147)]

To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!